In [23]:
from mxnet.gluon import nn

#Alexnet
net = nn.Sequential()
with net.name_scope():
        # 第一阶段
    #net.add(nn.Conv2D(channels=96, kernel_size=11, strides=4, activation='relu'))
    net.add(nn.Conv2D(channels=96, kernel_size=11, strides=4))
    net.add(nn.BatchNorm(axis=1))
    net.add(nn.Activation(activation='relu'))
    net.add(nn.MaxPool2D(pool_size=3, strides=2))
        # 第二阶段
    #net.add(nn.Conv2D(channels=256, kernel_size=5, padding=2, activation='relu'))
    net.add(nn.Conv2D(channels=256, kernel_size=5, padding=2,))
    net.add(nn.BatchNorm(axis=1))
    net.add(nn.Activation(activation='relu'))
    net.add(nn.MaxPool2D(pool_size=3, strides=2))
        # 第三阶段
    net.add(nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'))
    net.add(nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'))
    net.add(nn.Conv2D(channels=256, kernel_size=3, padding=1, activation='relu'))
    net.add(nn.MaxPool2D(pool_size=3, strides=2))
        # 第四阶段
    net.add(nn.Flatten())
    net.add(nn.Dense(4096, activation="relu"))
    net.add(nn.Dropout(.5))
        # 第五阶段
    net.add(nn.Dense(4096, activation="relu"))
    net.add(nn.Dropout(.5))
        # 第六阶段
    net.add(nn.Dense(10))

In [24]:
import sys
sys.path.append('..')
import utils

#Alexnet使用Imagenet数据，其中输入图片大小一般是224×224
train_data, test_data = utils.load_data_fashion_mnist(
    batch_size=64, resize=224)

In [22]:
from mxnet import ndarray as nd
class MyInit(init.Initializer):
    def __init__(self):
        super(MyInit, self).__init__()
        self._verbose = True
    def _init_weight(self, _, arr):
        # 初始化权重，使用out=arr后我们不需指定形状
        print('init weight', arr.shape)
        nd.random_normal(loc=0, scale=1, out=arr)
    def _init_bias(self, _, arr):
        print('init bias', arr.shape)
        # 初始化偏移
        arr[:] = 2
net.initialize(ctx=ctx, init=MyInit())


In [25]:
from mxnet import init
from mxnet import gluon

ctx = utils.try_gpu()
#myinit = init.Normal() #
#net.initialize(ctx=ctx, init=init.Xavier())
net.initialize(ctx=ctx)
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.01})
utils.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=5)

Epoch 0. Loss: 765041841073876090507856183296.000000, Train acc 0.098914, Test acc 0.100119
Epoch 1. Loss: 2.302649, Train acc 0.096948, Test acc 0.100119
Epoch 2. Loss: 2.302646, Train acc 0.097248, Test acc 0.099821
Epoch 3. Loss: 2.302646, Train acc 0.098647, Test acc 0.100119
Epoch 4. Loss: 2.302648, Train acc 0.095432, Test acc 0.100717
